<a href="https://colab.research.google.com/github/isamdr86/towards-ai/blob/main/notebooks/10-Adding_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables


In [1]:
!pip install -q llama-index==0.10.57 openai==1.37.0 llama-index-finetuning llama-index-postprocessor-cohere-rerank llama-index-embeddings-huggingface llama-index-embeddings-cohere cohere==5.6.2 tiktoken==0.7.0 chromadb==0.5.5  llama-index-vector-stores-chroma==0.1.10 llama-index-llms-gemini==0.1.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.6 MB/s eta 

In [2]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 tiktoken==0.7.0 --force-reinstall --quiet

In [3]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')
os.environ["CO_API_KEY"] = userdata.get('co_api_key')
cohere_key = os.environ["CO_API_KEY"]

In [4]:
# Allows running asyncio in environments with an existing event loop, like Jupyter notebooks.

import nest_asyncio

nest_asyncio.apply()

# Load a Model


In [5]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=1, model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

### Download vector store

In [6]:
from huggingface_hub import hf_hub_download

vectorstore = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="vectorstore.zip", repo_type="dataset", local_dir=".")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vectorstore.zip:   0%|          | 0.00/97.2M [00:00<?, ?B/s]

In [7]:
!unzip -o vectorstore.zip

Archive:  vectorstore.zip
   creating: ai_tutor_knowledge/
   creating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/length.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/index_metadata.pickle  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/link_lists.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/header.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/data_level0.bin  
  inflating: ai_tutor_knowledge/chroma.sqlite3  


In [8]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex

# Create the vector index
db = chromadb.PersistentClient(path="./ai_tutor_knowledge")
chroma_collection = db.get_or_create_collection("ai_tutor_knowledge")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(vector_store)

# Query Dataset


In [9]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

# Define the Cohere Reranking object to return only the first two highest ranking chunks.

#cohere_rerank = CohereRerank(top_n=2, api_key=cohere_key)
cohere_rerank3 = CohereRerank(top_n=2, api_key=cohere_key,model = 'rerank-english-v3.0')

In [10]:
# Define the ServiceCotext object to tie the LLM for generating final answer,
# and the embedding model to help with retrieving related nodes.
# The `node_postprocessors` function will be applied to the retrieved nodes.
query_engine = index.as_query_engine(
    similarity_top_k=5, node_postprocessors=[cohere_rerank3]
)

res = query_engine.query("Write about Retrieval Augmented Generation?")

In [11]:
res.response

"Retrieval-Augmented Generation (RAG) is an advanced approach in natural language processing that enhances the capabilities of generative models by integrating external knowledge sources. RAG systems consist of two primary components: Retrieval and Generation.\n\nThe retrieval component focuses on extracting relevant information from various external knowledge bases through two key phases: indexing and searching. Indexing organizes documents to facilitate efficient retrieval through methods like inverted indexes for sparse retrieval or dense vector encoding for dense retrieval. The searching phase utilizes these indexes to fetch relevant documents based on user queries, often enhanced by rerankers to optimize the order of the retrieved documents.\n\nThe generation component takes the information retrieved along with user queries to generate coherent, contextually relevant responses. It employs prompting methods such as Chain of Thought and Tree of Thought, which help to improve the qua

In [12]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 1c324686-fad7-4a41-bfd3-d44f9612ca91
Title	 Evaluation of Retrieval-Augmented Generation: A Survey:Research Paper Information
Text	 Authors: Hao Yu, Aoran Gan, Kai Zhang, Shiwei Tong, Qi Liu, and Zhaofeng Liu.Numerous studies of Retrieval-Augmented Generation (RAG) systems have emerged from various perspectives since the advent of Large Language Models (LLMs). The RAG system comprises two primary components: Retrieval and Generation. The retrieval component aims to extract relevant information from various external knowledge sources. It involves two main phases, indexing and searching. Indexing organizes documents to facilitate efficient retrieval, using either inverted indexes for sparse retrieval or dense vector encoding for dense retrieval. The searching component utilizes these indexes to fetch relevant documents based on the user's query, often incorporating the optional rerankers to refine the ranking of the retrieved documents. The generation component utilizes the retr

# Evaluate Cohere rerank

In [14]:
from huggingface_hub import hf_hub_download
from llama_index.finetuning.embeddings.common import (
    EmbeddingQAFinetuneDataset,
)

# Download the evaluation dataset
hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="rag_eval_dataset_question_context_subset_50.json", repo_type="dataset", local_dir=".")
rag_eval_dataset = EmbeddingQAFinetuneDataset.from_json("./rag_eval_dataset_question_context_subset_50.json")

(…)_dataset_question_context_subset_50.json:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

In [15]:
import pandas as pd


#  A simple function to show the evaluation result.
def display_results_retriever(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Retriever Name": [name], "Hit Rate": [hit_rate], "MRR": [mrr]}
    )

    return metric_df

In [16]:
from llama_index.core.evaluation import RetrieverEvaluator

# We can evaluate the retievers with different top_k values.
for i in [2, 4, 6, 8, 10]:
    retriever = index.as_retriever(
        similarity_top_k=i, node_postprocessors=[cohere_rerank3]
    )
    retriever_evaluator = RetrieverEvaluator.from_metric_names(
        ["mrr", "hit_rate"], retriever=retriever
    )
    eval_results = await retriever_evaluator.aevaluate_dataset(rag_eval_dataset)
    print(display_results_retriever(f"Retriever top_{i}", eval_results))

    Retriever Name  Hit Rate   MRR
0  Retriever top_2       0.6  0.57
    Retriever Name  Hit Rate       MRR
0  Retriever top_4      0.68  0.591667
    Retriever Name  Hit Rate       MRR
0  Retriever top_6      0.68  0.591667
    Retriever Name  Hit Rate       MRR
0  Retriever top_8      0.74  0.599881
     Retriever Name  Hit Rate       MRR
0  Retriever top_10      0.76  0.602103
